In [23]:
# Importaciones necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [24]:
# Echemos un vistazo al fichero
!cat "Ficheros/Manchas/SN_d_tot_V2.0.csv" | head

1818;01;01;1818.001;  -1; -1.0;   0;1
1818;01;02;1818.004;  -1; -1.0;   0;1
1818;01;03;1818.007;  -1; -1.0;   0;1
1818;01;04;1818.010;  -1; -1.0;   0;1
1818;01;05;1818.012;  -1; -1.0;   0;1
1818;01;06;1818.015;  -1; -1.0;   0;1
1818;01;07;1818.018;  -1; -1.0;   0;1
1818;01;08;1818.021;  65; 10.2;   1;1
1818;01;09;1818.023;  -1; -1.0;   0;1
1818;01;10;1818.026;  -1; -1.0;   0;1
cat: write error: Broken pipe


In [25]:
# Con la descripción de los datos que tenemos (ver "SN_d_tot_V2.0_description.txt") y 
# sabiendo que los NaN se coresponden con el valor -1 (y que también tenemos 0) podemos ya cargarlo:
columns = ['Year', 'Month', 'Day', 'Fraction', 'Spots', 'Stdev', 'Observ', 'Def/Prov']
df = pd.read_csv('Ficheros/Manchas/SN_d_tot_V2.0.csv', sep = ';', header = None, names = columns, na_values = [0, -1])
df

,Year,Month,Day,Fraction,Spots,Stdev,Observ,Def/Prov
0,1818,1,1,1818.001,-1,NaN,0,1.0
1,1818,1,2,1818.004,-1,NaN,0,1.0
2,1818,1,3,1818.007,-1,NaN,0,1.0
3,1818,1,4,1818.010,-1,NaN,0,1.0
4,1818,1,5,1818.012,-1,NaN,0,1.0
...,...,...,...,...,...,...,...,...
73834,2020,2,25,2020.152,0,NaN,25,NaN
73835,2020,2,26,2020.154,0,NaN,30,NaN
73836,2020,2,27,2020.157,0,NaN,28,NaN
73837,2020,2,28,2020.160,0,NaN,32,NaN


In [26]:
# No parece que nos haya transformado en NaN los datos de 'Spots', debe haber espacios en esa columna,
# vamos a quitarlos
columns = ['Year', 'Month', 'Day', 'Fraction', 'Spots', 'Stdev', 'Observ', 'Def/Prov']
df = pd.read_csv('Ficheros/Manchas/SN_d_tot_V2.0.csv', sep = r'\s*;\s*', header = None, names = columns, na_values =[0, -1],
                engine = 'python')
df

,Year,Month,Day,Fraction,Spots,Stdev,Observ,Def/Prov
0,1818,1,1,1818.001,NaN,NaN,NaN,1.0
1,1818,1,2,1818.004,NaN,NaN,NaN,1.0
2,1818,1,3,1818.007,NaN,NaN,NaN,1.0
3,1818,1,4,1818.010,NaN,NaN,NaN,1.0
4,1818,1,5,1818.012,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
73834,2020,2,25,2020.152,NaN,NaN,25.0,NaN
73835,2020,2,26,2020.154,NaN,NaN,30.0,NaN
73836,2020,2,27,2020.157,NaN,NaN,28.0,NaN
73837,2020,2,28,2020.160,NaN,NaN,32.0,NaN


In [27]:
# Un poco de información
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73839 entries, 0 to 73838
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      73839 non-null  int64  
 1   Month     73839 non-null  int64  
 2   Day       73839 non-null  int64  
 3   Fraction  73839 non-null  float64
 4   Spots     59396 non-null  float64
 5   Stdev     67975 non-null  float64
 6   Observ    70592 non-null  float64
 7   Def/Prov  73687 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 4.5 MB


In [28]:
# Borramos las fechas anteriores a 1975, puesto que no nos interesan para el trabajo posterior
df.drop(df[df['Year'] < 1975].index, inplace = True)

In [29]:
# Controlamos los NaN
df.isna().sum()


Year           0
Month          0
Day            0
Fraction       0
Spots       2270
Stdev        875
Observ         0
Def/Prov     152
dtype: int64

In [30]:
# Para evitar eliminar registros usaremos interpolación para rellenar los NaN
df['Spots'].interpolate(method = 'linear', limit_direction = 'forward')

57343    43.0
57344    43.0
57345    40.0
57346    28.0
57347    26.0
         ... 
73834    11.0
73835    11.0
73836    11.0
73837    11.0
73838    11.0
Name: Spots, Length: 16496, dtype: float64

In [31]:
# Crearemos una columna en formato fecha para operaciones posteriores
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [32]:
# Tomaremos como outliers la media + 3Stdev (~325)
len(df[df['Spots'] > 325])

114

In [33]:
# Los borramos
df.drop(df[df.Spots > 325].index, inplace =True)

In [34]:
# Solo nos interesan la nueva columna de fecha y el número de manchas (Spots). Eliminamos el resto
df.drop(['Year', 'Month', 'Day', 'Fraction', 'Stdev', 'Observ', 'Def/Prov'], axis = 'columns', inplace = True)

In [35]:
# Reordenamos
df = df[['Date', 'Spots']]

In [36]:
# Lo guardamos 
df.to_csv('Spots_new.csv', index = False)